In [8]:
# First cell - imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import config
import time
import json
import csv
import os
from bs4 import BeautifulSoup

# Run this cell first

In [9]:
# Second cell - initialize driver
driver = webdriver.Chrome()

In [10]:
# Third cell - login
def login():
    driver.get("https://www.instagram.com")
    
    username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
    password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))
    
    username.clear()
    username.send_keys(config.username2)
    password.clear()
    password.send_keys(config.password2)
    
    button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']")))
    button.click()

# Run login when ready
login()

In [11]:
#Go to profile
profile_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"a[href*='{config.username2}']")))
profile_button.click()

#Go to following
profile_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"a[href*='/{config.username2}/following']")))
profile_button.click()


In [12]:
#Scroll down to load all following
modal = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "div[class='xyi19xy x1ccrb07 xtf3nb5 x1pc53ja x1lliihq x1iyjqo2 xs83m0k xz65tgg x1rife3k x1n2onr6']"))
)

last_height = driver.execute_script("return arguments[0].scrollHeight", modal)
while True:
    # Scroll to bottom
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", modal)
    time.sleep(2)
    
    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return arguments[0].scrollHeight", modal)
    if new_height == last_height:
        break
    last_height = new_height

In [13]:
soup = BeautifulSoup(driver.page_source, "html.parser")

#First find the dialog div
dialog = soup.find('div', {'role': 'dialog'})

#Then find all the links within the dialog
accounts = dialog.find_all('a', 
    class_ = "x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz notranslate _a6hd"
    )

unique_accounts = {account['href'].lstrip('/').split('/?')[0] for account in accounts}  # Convert to set to remove duplicates


# If you need it as a list
unique_accounts_list = list(unique_accounts)

#sort the accounts by name
unique_accounts = sorted(unique_accounts, key=str.lower)

# Print unique hrefs (optional)
for account in unique_accounts:
    print(account)



chang.10.25
fwenclub
gigi_swy
horosocial_club
ivyyy.ng
natalieheha
t_toffler.l
virgo.together
weijin_huang
yiminghay


In [14]:
# Save to CSV file
base_dir = 'scraped_data'
csv_file = os.path.join(base_dir, 'instagram_accounts.csv')
headers = ['account', 'last_scrapped', 'num_images', 'num_videos']

with open(csv_file, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=headers)
    writer.writeheader()
    for username in unique_accounts:
        writer.writerow({
            'account': username,
            'last_scrapped': 0,
            'num_images': 0,
            'num_videos': 0
        })